In [12]:
import urllib.request
import os

os.makedirs("images", exist_ok=True)

img_urls = {
    "image1.jpg": "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/box.png",
    "image2.jpg": "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/box_in_scene.png"
}

for filename, url in img_urls.items():
    urllib.request.urlretrieve(url, f"images/{filename}")


In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths
img1_path = 'images/image1.jpg'
img2_path = 'images/image2.jpg'
output_match_path = 'output/matched_keypoints.jpg'
output_align_path = 'output/aligned_image.jpg'

# Load images
img1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
img2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)

# Convert to grayscale
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Detect ORB keypoints and descriptors
orb = cv2.ORB_create(5000)
kp1, des1 = orb.detectAndCompute(gray1, None)
kp2, des2 = orb.detectAndCompute(gray2, None)

# Match descriptors using BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

# Sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw matches
matched_img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=2)
cv2.imwrite(output_match_path, matched_img)

# Extract location of good matches
pts1 = np.float32([kp1[m.queryIdx].pt for m in matches[:50]]).reshape(-1, 1, 2)
pts2 = np.float32([kp2[m.trainIdx].pt for m in matches[:50]]).reshape(-1, 1, 2)

# Compute homography
H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)

# Warp image2 to image1
height, width = img1.shape[:2]
aligned_img = cv2.warpPerspective(img2, H, (width, height))
cv2.imwrite(output_align_path, aligned_img)

print("Matching and alignment complete.")


Matching and alignment complete.
